#### Imports

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import json
from newspaper import Article
from datetime import datetime

In [116]:
members = ['mario draghi', 'vitor constancio', 'benoit coeure', 'yves mersche', 'sabine lautenschlager', 'peter praet']
# representatives = ['luc coene', 'jens weidmann', 'patrick honohan', 'yannis stournaras', 'lui maria linde', 'ardo hansson',
#                    'christian noyer', 'ignazio visco', ]

#### Grab homepages, search from there
start:
- project Syndicate
- WorldCrunch
- Eurozine

In [39]:
sites = ['https://www.project-syndicate.org/', 'http://www.worldcrunch.com/business-finance', 'http://www.eurozine.com/']
r = requests.get(sites[1])
worldcrunch_soup = BeautifulSoup(r.text, 'lxml')

In [40]:
links = worldcrunch_soup.find_all('a')

In [41]:
title = re.compile('[A-Z][\w-]')

article_links = []

no_match = 0

for link in links:
    if title.match(link.text.strip()):
        article_links.append(link)
    else:
        no_match += 1
        
print('Number of links : {}'.format(str(len(article_links))))
print('Other links: {}'.format(str(no_match)))
        

Number of links : 50
Other links: 86


## NOTE: have to do this multiple times to get to 29 links

In [44]:
# stops = ['SUBSCRIBE', 'subscribe', 'Terms of Use', 'About Us', 'Privacy Policy']

for link in article_links:
    if len(link.text.split()) < 3:
        article_links.remove(link)
    elif '#' in str(link):
        article_links.remove(link)

print(len(article_links))

29


#### use newspaper to extract articles from remaining links

In [46]:
base_url = 'http://www.worldcrunch.com/'

def parseArticle(base_url, article_links):
    
    worldcrunch_dic = {}

    for link in article_links[2:]:
        url = base_url+link['href']
        r = requests.get(url)
        # Beautiful Soup
        soup = BeautifulSoup(r.text, 'lxml')

        try:
            publish_date = soup.find("meta", {"name":"DISPLAYDATE"})['content']
            r_datetime = datetime.strptime(publish_date, '%Y-%m-%d %H:%M:%S')
            d = datetime.strftime(r_datetime, '%Y-%m-%d')
            t = datetime.strftime(r_datetime, '%H:%M:%S')
            author = soup.find_all("p", {"class":"resize-auth"})[0].find('a').text.strip()
            raw_text = soup.find("div", {"class":"article-text"}).text
            if d in worldcrunch_dic:
                worldcrunch_dic[d].append([author, t, raw_text])
            else:
                worldcrunch_dic[d] = [(author, t, raw_text)]
        except:
            pass
    return worldcrunch_dic

worldcrunch_dic = parseArticle(base_url, article_links)

In [48]:

with open('worldcrunch.json', 'w') as fp:
    json.dump(worldcrunch_dic, fp)

In [50]:
len(worldcrunch_dic.keys())

22

#### Try methods for syndicate

In [192]:
search_url = 'https://www.project-syndicate.org/search?query=mario%20draghi&language=english&sortBy=publishedon&sortOrder=desc'
base_url = 'https://www.project-syndicate.org'

In [177]:
r = requests.get(search_url)
soup = BeautifulSoup(r.text, 'lxml')
article_links = soup.find_all("section", {"class":"small-12 medium-12 large-7 columns search-results listing authors"})[0].find_all('article', {"class":"section economics "})

In [77]:
def NewspaperParse(url, authors='authors'):
    article = Article(url)   
    article.download()
    article.parse()
    r_datetime = article.publish_date
    d = datetime.strftime(r_datetime, '%Y-%m-%d')
    t = datetime.strftime(r_datetime, '%H:%M:%S')
    raw_text = article.text
    if authors =='authors':
        author = article.authors[0]
    else:
        author = 'none'
    
    return d, t, raw_text, author 

def PS_parse(base_url, article_links):
    ps_dic = {}
    for link in article_links:
        ext = link.find("h2").find('a')['href']
        url = base_url + ext
        dt, raw_text, author = NewspaperParse(url)
        ps_dic[dt] = (author, raw_text)
    return ps_dic

# ps_dic = PS_parse(base_url, article_links)

In [310]:
with open('ps.json', 'w') as fp:
    json.dump(ps_dic, fp)

### BBC News

In [231]:
search_url = 'http://www.bbc.com/news/business'
base_url ='http://www.bbc.com'

r = requests.get(search_url)
soup = BeautifulSoup(r.text, 'lxml')

title_ext = soup.find('div',{"class":"column--primary"}).find('a', {"class":"title-link"})['href']


'/news/business-38507368'

In [237]:
# soup.find_all('div', {"class":"pigeon"})

In [187]:
for i in range(2,10):
    search_url = 'http://www.bbc.co.uk/search?q=mario+draghi&page={}'.format(i)
    r = requests.get(search_url)
    soup = BeautifulSoup(r.text, 'lxml')
    
    results = soup.find_all("section", {"id":"search-content"})
    links = []
    for r in results[0].find_all("li"):
        if r.find("a")['href'] not in article_links:
            links.append(r.find("a")['href'])
    
    article_links.extend(links)


In [189]:
# article_links

In [192]:
art = Article(article_links[1])
art.download()
art.parse()


'Image copyright Getty Images\n\nEurozone inflation has surged to its highest rate in more than three years, driven by increased prices for energy, food, alcohol and tobacco.\n\nThe annual inflation rate hit 1.1% last month, according to official statistics agency Eurostat, a sharp jump from November\'s rate of 0.6%.\n\nThe rate is the highest since September 2013, when inflation was also 1.1%.\n\nThe higher-than-expected increase brings inflation closer to the European Central Bank\'s target of just below 2%.\n\nECB chief Mario Draghi has said he expects inflation to reach the target by 2018 or 2019.\n\nLast month\'s increase was driven mainly by a jump in energy prices, which rose by 2.5% year-on-year in December, their first increase in over a year. Energy prices were boosted by oil cartel Opec\'s decision to cut output.\n\nFood, alcohol and tobacco prices rose 1.2% year-on-year, while services were also 1.2% more expensive than a year ago.\n\nShort-lived rise?\n\nThe rise will help

In [194]:
art.authors

[]

In [165]:
soup = BeautifulSoup(r.text, 'html.parser')

search_content = soup.find("section", {"class":"search-content"})
for ol in search_content.find_all("ol"):
    print(ol.find("li"))

<li data-result-number="1">
<article class="has_image media-text" itemscope="">
<aside class="flags top">
<dl>
<dt>Published Date</dt>
<dd>
<time class="display-date" datetime="2016-11-28T13:21:50Z">
           28 Nov 2016       </time>
</dd>
</dl>
</aside>
<a href="http://www.bbc.co.uk/news/live/uk-politics-parliaments-38132872" id="search-result-0">
<picture>
<img alt="" class="" src="http://ichef.bbci.co.uk/news/200/cpsprodpb/0347/production/_92693800_draghiap.jpg" srcset="http://ichef.bbci.co.uk/news/200/cpsprodpb/0347/production/_92693800_draghiap.jpg"/> </picture>
</a>
<aside class="flags mid">
<dl>
<dt>Published Date</dt>
<dd>
<time class="display-date" datetime="2016-11-28T13:21:50Z">
           28 Nov 2016       </time>
</dd>
</dl>
</aside>
<div>
<h1 itemprop="headline"><a href="http://www.bbc.co.uk/news/live/uk-politics-parliaments-38132872">MEPs question Mario Draghi</a></h1>
<p class="summary short">…European Central Bank (ECB) chief <em>Mario</em> <em>Draghi</em> appeared 

### Gaurdian


In [293]:
import json

with open('guardianresults.json') as data_file:    
    guardian_results = json.load(data_file)

In [303]:
guardian_dic = {}

for result in guardian_results['response']['results']:
    if result['type'] == 'article':
        url = result['webUrl']
        
        article = Article(url)
        article.download()
        article.parse()
        raw_text = article.text
        author = article.authors
        r_datetime = article.publish_date
        d = datetime.strftime(r_datetime, '%Y-%m-%d')
        t = datetime.strftime(r_datetime, '%H:%M:%S')
        if d in guardian_dic:
            guardian_dic[d].extend([(author, t, raw_text)])
        else:
            guardian_dic[d] = [(author, t, raw_text)]
        

In [307]:
guardian_dic[list(guardian_dic.keys())[3]]

[(['Steve Boxer'],
  '00:00:00',
  'We can assume that David Cameron is a bit of a gamer, having been busted obsessively playing Fruit Ninja and Angry Birds in the past. So there is an outside chance that, finally, he might soften his government’s harsh, grudging response to the Syrian refugee crisis. Because one of the gaming world’s most famous characters – the moustachioed plumber Mario – has been co-opted into highlighting the plight of those attempting to flee from the Scylla and Charybdis of Isis and Bashar al-Assad.\n\nA Syrian artist, pseudonymously known as Samir al-Mutfi, has created Syrian Super Mario, a satirical video that is currently going viral, highlighting the horrors faced by Syrian refugees – by tweaking the 1985 video game Super Mario Bros. Mario has taken many forms over the decades, but this time he has become Refugee Mario. And instead of making his way to Bowser’s castle in order to rescue Princess Peach, he merely has to get to the castle, which now bears the 

In [309]:
with open('guardian.json', 'w') as fp:
    json.dump(guardian_dic, fp)

### The FT - need to get corporate developer license

In [6]:
# search_url = 'https://www.ft.com/search?q=mario+draghi'

# r = requests.get(search_url)
# soup = BeautifulSoup(r.text, 'lxml')

for li in soup.find_all('li', {"class": "o-teaser-collection__item o-grid-row"}):
    article_links.extend(li.find_all('a'))
    
    

In [22]:
link = article_links[1]['href']

In [13]:
base_url = 'http://ft.com' 

In [ ]:
def ParseArticles(base_url, article_links):
    for link in article_links:
        url = base_url + link
        r = requests.get(url)

In [23]:
url = base_url + link

In [34]:
article = Article(url)

In [35]:
article.download()
article.parse()
article.publish_date

In [36]:
article.text

'Subscribe to FT.com\n\nSubscribe to read: Italy’s president picks up pieces after political crisis\n\nAlready a subscriber? Sign in here'

### EurActiv - has no authors on articles

In [90]:
soup = BeautifulSoup(requests.get('http://www.euractiv.com/?s=mario+draghi').text)

/Users/hannahdeen/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [97]:
block = soup.find_all("div",{"class":"row"})
article_links = []
    
for b in block.find_all("a"):
    if len(b['href'].split('/')) > 6 and b['href'] not in article_links:
        article_links.append(b['href'])

In [112]:
search_url = 'http://www.euractiv.com/?s=mario+draghi'
# 'http://www.euractiv.com/page/2/?s=mario+draghi'
# for all names, and 10 pages?

def getLinksEurActiv(search_url):
    r = requests.get(search_url)
    soup = BeautifulSoup(r.text, 'lxml')
    
    block = soup.find_all("div",{"class":"row"})[4]
    article_links = []
    
    for b in block.find_all("a"):
        if len(b['href'].split('/')) > 6 and b['href'] not in article_links:
            article_links.append(b['href'])
    
    return article_links


def NewspaperParse(url, authors='authors'):
    article = Article(url)   
    
    try:
        article.download()
        article.parse()
        r_datetime = article.publish_date
        d = datetime.strftime(r_datetime, '%Y-%m-%d')
        t = datetime.strftime(r_datetime, '%H:%M:%S')
        raw_text = article.text
        if authors =='authors':
            author = article.authors[0]
        else:
            author = 'none'
        return d, t, raw_text, author
    except:
        return 'none', 'none', 'none', 'none'
        

article_links = []
euractiv_dic = {}

for i in range(2,10):
    search_url = 'http://www.euractiv.com/page/{}/?s=mario+draghi'.format(i)
    print(search_url)
    links = getLinksEurActiv(search_url)
    article_links.extend(links)
    article_links = list(set(article_links))
    
    for link in article_links:
        d, t, raw_text, author = NewspaperParse(link, 'none')
        if d != 'none':
            if d in euractiv_dic:
                euractiv_dic[d].append([(author, t, raw_text)])
            else:
                euractiv_dic[d] = [(author, t, raw_text)]
    



http://www.euractiv.com/page/2/?s=mario+draghi
http://www.euractiv.com/page/3/?s=mario+draghi
http://www.euractiv.com/page/4/?s=mario+draghi
http://www.euractiv.com/page/5/?s=mario+draghi
http://www.euractiv.com/page/6/?s=mario+draghi
http://www.euractiv.com/page/7/?s=mario+draghi
http://www.euractiv.com/page/8/?s=mario+draghi
You must `download()` an article before calling `parse()` on it!
http://www.euractiv.com/page/9/?s=mario+draghi
You must `download()` an article before calling `parse()` on it!


In [118]:
'http://content.guardianapis.com/search?section=business&from-date=2001-01-01&page-size=161&q=vitor%20constancio&api-key=458f61b9-2ff0-4a3f-98a1-feff65660ca6'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31656  100 31656    0     0  38823      0 --:--:-- --:--:-- --:--:-- 38794


In [ ]:
class NewspaperScraper():

	def newspaper(url, authors='authors'):
		article = Article(url)   

		try:
			article.download()
			article.parse()
			r_datetime = article.publish_date
			d = datetime.strftime(r_datetime, '%Y-%m-%d')
			t = datetime.strftime(r_datetime, '%H:%M:%S')
			raw_text = article.text
			if authors =='authors':
			    author = article.authors[0]
			else:
			    author = 'none'
		return d, t, raw_text, author
	except:
	return 'none', 'none', 'none', 'none'


	def dictify(dic, d, t, author, raw_text):
		if d in dic:
			dic[d].extend([(author, t, raw_text)])
		else:
			dic[d] = [(author, t, raw_text)]
		return dic	


class Guardian(NewspaperScraper):
	
	base_url = 'http://www.theguardian.com' # same for every instance
	
	def __init__(self):
		self.dic = {} # creates a new empty dic for each Guardian class

	def getlinks(self, search_url):
		#  uses the api unlike others

	def parse(self, base_url=None, article_links):
		
		if article_links = None:
			raise Exception.message('No links! Run getlinks() before parse()')

		for link in article links:
			d, t, author, raw_text = newspaper(link)
			self.dictify(self.dic, d, t, author, raw_text)

		return self.dic

In [119]:
r = requests.get('http://content.guardianapis.com/search?section=business&from-date=2001-01-01&page-size=161&q=vitor%20constancio&api-key=458f61b9-2ff0-4a3f-98a1-feff65660ca6')

In [169]:
# r.text

In [168]:
# json.loads(r.text)['response']

In [172]:
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
# import time

# firefox = webdriver.Firefox()

# gmail_url = "https://mail.google.com"

# firefox.get(gmail_url)

In [11]:
from newses import Guardian

In [12]:
guardian = Guardian()

In [13]:
guardian.getlinks('mario draghi')

['https://www.theguardian.com/business/2016/nov/28/ecbs-mario-draghi-urges-uk-to-disclose-brexit-plans',
 'https://www.theguardian.com/business/2016/jun/21/mario-draghi-janet-yellen-poised-eu-referendum-fallout',
 'https://www.theguardian.com/business/2016/apr/21/mario-draghi-defends-ecb-independence-german-criticism-central-bank-eurozone-interest-rates',
 'https://www.theguardian.com/world/2016/mar/11/mario-draghi-reducing-youth-unemployment-is-a-priority-for-everyone',
 'https://www.theguardian.com/business/2016/feb/15/stock-markets-rally-mario-draghi-qe-stimulus-negative-rates',
 'https://www.theguardian.com/business/2015/dec/03/mario-draghis-big-bazooka-turns-peashooter-eurozone-qe-ecb',
 'https://www.theguardian.com/business/2016/jan/22/mario-draghi-global-recession-fears-refugee-opportunity',
 'https://www.theguardian.com/business/2015/sep/03/mario-draghi-ready-to-prop-up-eurozone-with-more-stimulus',
 'https://www.theguardian.com/business/2015/oct/22/mario-draghi-ecb-prepared-to

In [9]:
from newspaper import Article
article = Article('http://www.bbc.co.uk/news/business-17471617')
article.download()
article.parse()

In [17]:
import requests
from bs4 import BeautifulSoup

r = requests.get('http://www.bbc.co.uk/news/business-17471617')
soup = BeautifulSoup(r.text, 'lxml')

In [22]:
import datetime
date = datetime.datetime.strptime(soup.find("div", {"class":"date date--v2"})['data-datetime'], '%d %B %Y')

In [23]:
datetime.datetime.strftime(date, '%Y-%m-%d')

'2012-03-22'

In [26]:
art = Article('https://www.theguardian.com/world/2016/mar/11/mario-draghi-reducing-youth-unemployment-is-a-priority-for-everyone')

In [27]:
art.download()
art.parse()

In [28]:
art.text

'What does the European Central Bank’s own research on incomes and younger generations say?\n\nDuring the great recession, the young [aged 16-34] lost about 5% of household income in real terms. That was of course substantial and had a big impact on the personal plans of many young people. But the magnitude was not so different from the losses suffered by other generations, although of course the young do not have the same assets and savings to fall back on in tough economic times. Even more worrying was the sharp increase in youth unemployment in Europe.\n\nEuropean job market is rigged against younger workers, says Draghi Read more\n\nAre you worried about the position of young adults in Europe and is this an increasingly urgent issue?\n\nYouth unemployment is a tragedy and prevents people from playing a full and meaningful part in society. If every second young person is out of work – as is still the case in some countries in Europe – it seriously harms the economy, because people w

In [29]:
art.authors

['Shiv Malik', 'Larry Elliott Economics Editor']

In [30]:
art.publish_date

datetime.datetime(2016, 3, 11, 0, 0)

In [33]:
lister = []

In [36]:
if len(lister):
    print('y')
else:
    print('n')

n
